In [ ]:
# Run the following two cells if you are using Google Colab
# Install Colab Anaconda (takes around 20 seconds)
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Install OSMnx (takes around 3 minutes)
!conda install -c conda-forge osmnx -y

# 15-minute city: calculating a specific location meets the criteria of a 15-minute city

In this lab, we will evaluate if a specific location meets the criteria of a 15-minute city. A 15-minute city is a concept that aims to create a city where all the necessary services and amenities are within a 15-minute walk or bike ride from home. The idea is to reduce the need for long commutes and make cities more sustainable and livable.

To be specific, we will retrieve Points of Interest (POIs) data from the OpenStreetMap database and calculate the distance from a specific location to the nearest POI in each category. We will then evaluate if the location meets the criteria of a 15-minute city based on the calculated distances.

Source: https://wiki.openstreetmap.org/wiki/Key:shop

## Import Packages
A Python package is a way of organizing related Python modules into a single directory hierarchy. It provides a mechanism for grouping Python code files, resources, and configuration settings in a structured manner, making it easier to manage and distribute code. They also facilitate code reuse and distribution by allowing developers to bundle related functionality together and share it with others.

The following packages are used in this notebook:<br>
`osmnx` is a Python package to retrieve, model, analyze, and visualize street networks from OpenStreetMap. <br>
source: https://osmnx.readthedocs.io/<br>

`folium` is a Python package to create interactive maps. <br>
source: https://python-visualization.github.io/folium/

The packages below were introduced in the previous notebook:<br>
`pandas` is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. <br>
source: https://pandas.pydata.org/docs/getting_started/overview.html

`geopandas` is the geographic expansion of `pandas`, allowing to have geometry and working with vector data. <br>
source: https://geopandas.org/en/stable/getting_started/introduction.html

`matplotlib` provides a collection of functions that make plots and maps. Each pyplot function makes some change to a figure: e.g., creates a figure, creates a plotting area in a figure, plots some lines in a plotting area, decorates the plot with labels, etc. <br>
source: https://matplotlib.org/stable/users/getting_started/

In [ ]:
# Import packages
import osmnx as ox
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

In [ ]:
# Change the directory of pyproj data
# This is necessary to avoid the error "PROJ data directory not found"
# when using pyproj with OSMnx

import pyproj
print(pyproj.datadir.get_data_dir())
pyproj.datadir.set_data_dir('/usr/local/lib/python3.11/dist-packages/pyproj/proj_dir/share/proj')
print(pyproj.datadir.get_data_dir())

In [ ]:
# Checking if pyproj is working
from pyproj import CRS
crs = CRS.from_epsg(4326)
print("✅ Checking CRS:", crs)

# Overview

# 1. Retrieve Points of Interest (POIs) from OpenStreetMap
##  1.1. Retrieve the Polygon of an administrative region

In [ ]:
# Load the geometry of Dongdaemun-gu, Seoul, South Korea
# aoi: Area of Interest
aoi = ox.geocode_to_gdf('Dongdaemun-gu, Seoul, South Korea')
aoi

In [ ]:
aoi.explore()

---
### *Exercise*
1. (2 points) Retrieve the Polygon of an administrative region (e.g., Dalseo-gu, Daegu) that you are interested in. <br>
You can get the official English name of an administrative region in the website (https://www.juso.go.kr/openIndexPage.do). <br>
The syntax is as shown below.

```python
    your_aoi = ox.geocode_to_gdf('Name of a Certain Addministrative Region')
    your_aoi.explore()
```
---

In [ ]:
# Your code here
your_aoi = ox.geocode_to_gdf('Name of a Certain Addministrative Region')
your_aoi.explore()

In [ ]:
""" Test code for the previous function.
This cell should NOT give any errors when it is run."""

assert type(your_aoi.loc[0, 'geometry']) == Polygon

print("Success!")

##  1.2. Retrieve Points of Interest (POIs) using OpenStreetMap Tags (Keys and Values)

Source: https://wiki.openstreetmap.org/wiki/Key:shop

In [ ]:
# Retrieve Point of Interest (POI) data from OpenStreetMap
food_gdf = ox.features_from_polygon(aoi.geometry[0], # Extent of the POI data
                                      tags={'shop': ['supermarket', 'grocery', 'deli', 'convenience']} # Tags to filter the POI data
                                      )
food_gdf

In [ ]:
# Clean the POI data
food_gdf = food_gdf[['shop', 'name', 'geometry']]
food_gdf

In [ ]:
food_gdf.explore()

---
### *Exercise*
2. (3 points) The following in the syntax of querying points of interest (POIs) from OpenStreetMap using the `osmnx` package. <br><br>
    ```python
    return_gdf = ox.features_from_polygon('Extent of the POI data', tags = {'OSM Tag Key': ['OSM Tag Value1', 'OSM Tag Value2', ...]})
    ```
    
    Find the transport-related POIs within the extent of Dongdaemun-gu (i.e., GeoDataFrame `aoi`) based on the key and value information below. <br>
    Then, save the return GeoDataFrame as `transport_gdf`.
    - Key: 'public_transport'
    - Values: 'station', 'platform'
    <br><br>

---

In [ ]:
# Your code here
transit_gdf = ox.features_from_polygon('Extent of the POI data',
                                         tags = {'OSM Tag Key': ['OSM Tag Value1', 'OSM Tag Value2', ...]
                                        })
transit_gdf

In [ ]:
""" Test code for the previous function.
This cell should NOT give any errors when it is run."""

assert 'public_transport' in transit_gdf.columns

print("Success!")

# 2. Automatically Retrieve POI data relevant to 15-minute cities <br>

In [ ]:
# Retrieve POI data relevant to 15-minute cities (i.e., food, medical, culture, education, transit, leisure)
'''
{'Category Name':
  {'OSM Tag Key': ['OSM Tag Value1', 'OSM Tag Value2', ...]}
 }
'''
poi_tags = {'food': {'shop': ['supermarket', 'grocery', 'deli', 'convenience']},
            'medical' : {'amenity': ['hospital', 'clinic', 'doctors']},
            'culture': {'amenity': ['community_centre', 'music_venue', 'social_centre', 'arts_centre', 'theatre', 'cinema']},
            'education': {'amenity': ['school', 'kindergarten', 'library']},
            'transit': {'public_transport': ['station', 'platform']},
            'leisure': {'leisure': ['park', 'playground']},
            }

In [ ]:
# Create a GeoDataFrame to store the POI data
poi_gdf = gpd.GeoDataFrame()

# Iterate over the categories and tags to retrieve the POI data
for category, tags in poi_tags.items():
    print(f'Retrieving {category} POI data...')
    print(tags)

    # Retrieve the POI data for each category
    temp_gdf = ox.features_from_polygon(aoi.geometry[0], tags=tags)

    # Clean the POI data
    temp_gdf = temp_gdf[['name', 'geometry']]

    # Add the category information to the POI data
    temp_gdf['category'] = category

    # Append the POI data to the GeoDataFrame
    poi_gdf = pd.concat([poi_gdf, temp_gdf])

    print("-------------------")

# Change the geometry of the POI data to the centroid of the geometry
poi_gdf['geometry'] = poi_gdf.apply(lambda x:x['geometry'].centroid, axis=1)
poi_gdf

In [ ]:
# Visualize the POI data
m = aoi.explore(facecolor='none', edgecolor='black', alpha=0.1)
poi_gdf.explore('category', legend=True, zorder=1, m=m)

---
### *Exercise*
3. (2 points) Run the following code blocks to retrieve 15-minute city related poi in your area of interest.<br>

```python
    # The code below will retrieve the POI data for your AOI
    # Retrieve POI data relevant to 15-minute cities (i.e., food, medical, culture, education, transit, leisure)

    poi_tags = {'food': {'shop': ['supermarket', 'grocery', 'deli', 'convenience']},
                'medical' : {'amenity': ['hospital', 'clinic', 'doctors']},
                'culture': {'amenity': ['community_centre', 'music_venue', 'social_centre', 'arts_centre', 'theatre', 'cinema']},
                'education': {'amenity': ['school', 'kindergarten', 'library']},
                'transit': {'public_transport': ['station', 'platform']},
                'leisure': {'leisure': ['park', 'playground']},
                }

    # Create a GeoDataFrame to store the POI data
    your_poi_gdf = gpd.GeoDataFrame()

    # Iterate over the categories and tags to retrieve the POI data
    for category, tags in poi_tags.items():  
        print(f'Retrieving {category} POI data...')
        print(tags)

        # Retrieve the POI data for each category
        temp_gdf = ox.features_from_polygon(your_aoi.geometry[0], tags=tags)

        # Clean the POI data
        temp_gdf = temp_gdf[['name', 'geometry']]

        # Add the category information to the POI data
        temp_gdf['category'] = category

        # Append the POI data to the GeoDataFrame
        your_poi_gdf = pd.concat([your_poi_gdf, temp_gdf])
        
    # Change the geometry of the POI data to the centroid of the geometry
    your_poi_gdf['geometry'] = your_poi_gdf.apply(lambda x:x['geometry'].centroid, axis=1)
    your_poi_gdf
```

---

In [ ]:
# The code below will retrieve the POI data for your AOI
# Retrieve POI data relevant to 15-minute cities (i.e., food, medical, culture, education, transit, leisure)

poi_tags = {'food': {'shop': ['supermarket', 'grocery', 'deli', 'convenience']},
            'medical' : {'amenity': ['hospital', 'clinic', 'doctors']},
            'culture': {'amenity': ['community_centre', 'music_venue', 'social_centre', 'arts_centre', 'theatre', 'cinema']},
            'education': {'amenity': ['school', 'kindergarten', 'library']},
            'transit': {'public_transport': ['station', 'platform']},
            'leisure': {'leisure': ['park', 'playground']},
            }

# Create a GeoDataFrame to store the POI data
your_poi_gdf = gpd.GeoDataFrame()

# Iterate over the categories and tags to retrieve the POI data
for category, tags in poi_tags.items():
    print(f'Retrieving {category} POI data...')
    print(tags)

    # Retrieve the POI data for each category
    temp_gdf = ox.features_from_polygon(your_aoi.geometry[0], tags=tags)

    # Clean the POI data
    temp_gdf = temp_gdf[['name', 'geometry']]

    # Add the category information to the POI data
    temp_gdf['category'] = category

    # Append the POI data to the GeoDataFrame
    your_poi_gdf = pd.concat([your_poi_gdf, temp_gdf])

# Change the geometry of the POI data to the centroid of the geometry
your_poi_gdf['geometry'] = your_poi_gdf.apply(lambda x:x['geometry'].centroid, axis=1)
your_poi_gdf

In [ ]:
# Visualize your POI data along with your AOI
m = your_aoi.explore(facecolor='none', edgecolor='black', alpha=0.1)
your_poi_gdf.explore('category', legend=True, zorder=1, m=m)

## Summary

The following is the summary of the code how to retrieve the Points of Interest (POIs) from OpenStreetMap (OSM) using the `osmnx` package. <br>

```python
# Retrieve the Polygon of an administrative region
aoi = ox.geocode_to_gdf(`Name of a Certain Addministrative Region`)

# We just iterated over the categories and tags to retrieve the POI data
poi_gdf = ox.features_from_polygon(`Extent of the POI data`
                                     tags={`Tag Key`: [`Tag Value 1`, `Tag Value 2` ...]}
                                     )
```

# 3. Query whether a specific location meets 15-minute city criteria

## 3.1. Define a speciic location

In [ ]:
# Geocoding is the process of converting addresses into geographic coordinates (latitude and longitude)
# Geocoding with the help of OpenStreetMap
khu_coordinates = ox.geocode('Kyung Hee University, Seoul, South Korea')
khu_coordinates

In [ ]:
# GeoDataFrame that contains the location information of Kyung Hee University (KHU)

# Manually input the coordinates of KHU
khu_df = pd.DataFrame({'name': ['KHU'],
                       'x': [127.05198672345011],
                       'y': [37.5971213]
                       })

# Automatically retrieve the coordinates of KHU
khu_df = pd.DataFrame({'name': ['KHU'],
                       'x': [khu_coordinates[1]],
                       'y': [khu_coordinates[0]]
                       })
khu_df

In [ ]:
# Convert the DataFrame to a GeoDataFrame
khu_gdf = gpd.GeoDataFrame(khu_df,
                           geometry=gpd.points_from_xy(khu_df.x, khu_df.y),
                           crs='EPSG:4326')
khu_gdf

In [ ]:
# Visualize the POI data
m = aoi.explore(facecolor='none', edgecolor='black', alpha=0.1)
poi_gdf.explore('category', legend=True, zorder=1, m=m)
folium.Marker(location=[khu_gdf.geometry.y, khu_gdf.geometry.x], popup='KHU').add_to(m)
m

---
### *Exercise*
4. (2 points) Create a DataFrame to store the coordinates of your specific location. <br>
- Get the coordinates information of a specific location within your Area of Interest (e.g., Dalseo-gu, Daegu) using the `geocode` function. <br>
- Enter the coordinates into DataFrame. If you don't like the geocoded location, you can manually enter the x and y coordinates of your location using Google Map. <br>
```python
    your_coordinates = ox.geocode('Address of your specific location')

    your_loc_df = pd.DataFrame({'name': ['Your Location'],
                            'x': [your_coordinates[1]], # You can update this if you don't like the retrieved coordinates
                            'y': [your_coordinates[0]]  # You can update this if you don't like the retrieved coordinates
                            })
    your_loc_df
```

---

In [ ]:
# Your code here
your_coordinates = ox.geocode('Address of your specific location')
print(your_coordinates)

your_loc_df = pd.DataFrame({'name': ['Your Location'],
                            'x': [your_coordinates[1]], # You can update this if you don't like the retrieved coordinates
                            'y': [your_coordinates[0]]  # You can update this if you don't like the retrieved coordinates
                            })
your_loc_df

In [ ]:
# Convert the DataFrame to a GeoDataFrame
your_loc_gdf = gpd.GeoDataFrame(your_loc_df,
                                geometry=gpd.points_from_xy(your_loc_df.x, your_loc_df.y),
                                crs='EPSG:4326')
your_loc_gdf

In [ ]:
# Visualize your POI data, your location, and your AOI
m = your_aoi.explore(facecolor='none', edgecolor='black', alpha=0.1)
your_poi_gdf.explore('category', legend=True, zorder=1, m=m)
folium.Marker(location=[your_loc_gdf.geometry.y, your_loc_gdf.geometry.x], popup='Your Location').add_to(m)
m

## 3.2. Change the coordinate system of the GeoDataFrame to UTM-K (EPSG:5179)

The unit of WGS84 (EPSG:4326) is degree, which is not suitable for distance calculation. Therefore, we need to change the coordinate system to UTM-K (EPSG:5179) to calculate the distance in meters.

In [ ]:
poi_gdf

In [ ]:
poi_gdf.crs

In [ ]:
khu_gdf

In [ ]:
khu_gdf.crs

In [ ]:
# You can use .to_crs() method to change the coordinate reference system (CRS) of a GeoDataFrame
poi_gdf_proj = poi_gdf.to_crs('EPSG:5179')
khu_gdf_proj = khu_gdf.to_crs('EPSG:5179')

In [ ]:
poi_gdf_proj.crs

In [ ]:
khu_gdf_proj.crs

---
### *Exercise*
5. (2 points) Convert the coordinate systems of your two GeoDataFrame (i.e., `your_poi_gdf` and `your_loc_gdf`) from WGS84 (EPSG:4326) to UTM-K (EPSG: 5179) <br>
    and then save the results to `your_poi_gdf_proj` and `your_loc_gdf_proj` respectively. <br>
    Syntax as shown below.

```python
    proj_gdf = gdf.to_crs(epsg = 'Provide EPSG Code of the Coordinate System here')
```
---

In [ ]:
# Your code here
your_poi_gdf_proj = your_poi_gdf.`DO SOMETHING HERE`
your_loc_gdf_proj = your_loc_gdf.`DO SOMETHING HERE`

In [ ]:
""" Test code for the previous function.
This cell should NOT give any errors when it is run."""

assert your_poi_gdf_proj.crs == 'EPSG:5179'
assert your_loc_gdf_proj.crs == 'EPSG:5179'

print("Success!")

## 3.3. Create a buffer around the specific location

Syntax to create a buffer around a point
```python
gdf = gdf.buffer(`RADIUS IN METERS`)

OR

gdf = gdf['geometry'].buffer(`RADIUS IN METERS`)
```

In [ ]:
khu_gdf_buffer = khu_gdf_proj.copy()
khu_gdf_buffer['geometry'] = khu_gdf_buffer.buffer(1000)
khu_gdf_buffer

In [ ]:
# Visualize the buffer
khu_gdf_buffer.explore()

---
### *Exercise*
6. (2 points) Create the buffer of your specific location (i.e., `your_loc_gdf_proj`) with a radius of 1000 meters. <br>
    Save the results to `your_loc_buffer`. <br>
    Syntax as shown below.

```python
    your_loc_buffer = your_loc_gdf_proj.copy()
    your_loc_buffer = your_loc_buffer.buffer('Radius in Meters')
    your_loc_buffer
```

---

In [ ]:
# Your code here
your_loc_buffer = your_loc_gdf_proj.copy()
your_loc_buffer['geometry'] = your_loc_buffer.buffer(`RADIUS IN METERS`)
your_loc_buffer


In [ ]:
""" Test code for the previous function.
This cell should NOT give any errors when it is run."""

assert type(your_loc_buffer.loc[0, 'geometry']) == Polygon
assert round(your_loc_buffer.loc[0, 'geometry'].area) == 3136548

print("Success!")

## 3.4. Select POIs within 1km of the specific location

Spatial operations methods in GeoPandas: <br>
- `intersects` method: Selects all rows that intersect a single geometry. <br>
- `within` method: Selects all rows that are within a single geometry. <br>
- `disjoint` method: Selects all rows that are disjoint to a single geometry. <br>

Syntax:
```python
`GEOMETRY COLUMN`.intersects(`A SINGLE GEOMETRY`) # intersect operation (can be replaced with within or disjoint)
```

Spatial operations in GeoPandas can be performed using the `loc` method. The loc method syntax is `df.loc[row condition, column condition]`. The following codes is the completed code to select a given geodataframe based on a spatial operation.<br>

Syntax:
```python
`SELECTED_GDF` = `GEODATAFRAME`.loc[`GEODATAFRAME_A`['geometry'].intersects(`A SINGLE GEOMETRY`)] # Intersects operation
```

Kinds of spatial operations: <br>
![](https://www.researchgate.net/profile/Tran-Dang-2/publication/286609202/figure/fig2/AS:925090808086531@1597570244506/Eight-OGC-spatial-relations-used-in-GeoXACML_W640.jpg)

In [ ]:
# Visualize the buffer and POIs within the AOI
fig, ax = plt.subplots(figsize=(10, 10))

poi_gdf_proj.plot('category', legend=True, ax=ax)
khu_gdf_buffer.boundary.plot(ax=ax, color='red')
plt.show()

In [ ]:
# Select buffer geometry
khu_gdf_buffer['geometry'][0]

In [ ]:
# Select POI data geometries
poi_gdf_proj['geometry']

In [ ]:
# Query the POI data which intersects the buffer
poi_gdf_proj.intersects(khu_gdf_buffer['geometry'][0])

In [ ]:
# Select the POI data within the buffer
poi_gdf_proj.loc[poi_gdf_proj.intersects(khu_gdf_buffer['geometry'][0])]

In [ ]:
# Assign the result into a new GeoDataFrame (poi_khu)
poi_khu_buffer = poi_gdf_proj.loc[poi_gdf_proj.intersects(khu_gdf_buffer['geometry'][0])]
poi_khu_buffer

In [ ]:
# Visualize the buffer and POIs within the AOI (Your case)
fig, ax = plt.subplots(figsize=(10, 10))

your_poi_gdf_proj.plot('category', legend=True, ax=ax)
your_loc_buffer.boundary.plot(ax=ax, color='red')
plt.show()

---
### *Exercise*
7. (2 points) Select the POI (i.e., `your_poi_gdf_proj`) within the 1km buffer of your specific location (i.e., your_loc_buffer) <br>
    and then save the results to `poi_your_buffer`. <br>
    Syntax as shown below.

```python
    poi_your_buffer = `GEODATAFRAME`.loc[`GEODATAFRAME_A`['geometry'].intersects(`A SINGLE GEOMETRY`)] # Intersects operation
    poi_your_buffer
```
---

In [ ]:
# Your code here
poi_your_buffer = your_poi_gdf_proj.loc[your_poi_gdf_proj.intersects(your_loc_buffer['geometry'][0])]
poi_your_buffer

# 3.5. Check the statistics of the POIs within 1km of the specific location

`DataFrame`['column'].value_counts() is a method to count the number of unique values in a column of a DataFrame. <br>

In [ ]:
# Check the number of POIs within the buffer (KHU case)
poi_khu_buffer['category'].value_counts()

In [ ]:
# Check the number of POIs within the buffer (your case)
poi_your_buffer['category'].value_counts()

# 4. Summary of the code

```python
# Get the Polygon of an administrative region
aoi = ox.geocode_to_gdf('Dongdaemun-gu, Seoul, South Korea')

# Retrieve the POI data for the AOI (iternation necessary)
poi_gdf = ox.features_from_polygon(`Extent of the POI data`
                                     tags={`Tag Key`: [`Tag Value 1`, `Tag Value 2` ...]}
                                     )

# Select a specific location to query 15-minute city criteria
your_coordinates = ox.geocode('Address of your specific location')

# Convert the coordinates to a DataFrame
your_loc_df = pd.DataFrame({'name': ['Your Location'],
                            'x': ['Enter the x-coordinate'],
                            'y': ['Enter the y-coordinate']
                            })

# Convert the DataFrame to a GeoDataFrame
your_loc_gdf = gpd.GeoDataFrame(your_loc_df,
                                geometry=gpd.points_from_xy(your_loc_df.x, your_loc_df.y),
                                crs='EPSG:4326')

# Convert the coordinate systems of the GeoDataFrames to UTM-K (EPSG:5179)
proj_gdf = gdf.to_crs(epsg = 'Provide EPSG Code of the Coordinate System here')

# Create a buffer around the specific location
your_loc_buffer = your_loc_buffer.buffer('Radius in Meters')

# Select POIs within 1km of the specific location
poi_your_buffer = `GEODATAFRAME`.loc[`GEODATAFRAME_A`['geometry'].intersects(`A SINGLE GEOMETRY`)] # Intersects operation

```

# Done